In [5]:
using Statistics 
using Plots
using LaTeXStrings
using DelimitedFiles
using StatsPlots

#Decimal to Bit Conversion

function invmap(a,L)
     map=zeros(Float64,2^L,3)
     w_d=zeros(Float64,L+1,binomial(L,Int(round(L/2))))
    for i in collect(0:1:L)
        j=1
        for k in collect(1:1:2^L)
            if h(ToBit(k-1,L),ToBit(0,L),L)==i
                w_d[i+1,j]=w[k]
                map[k,:]=[k,i+1,j]
                j+=1      
            end
        end
    end
    k=0
    check=0
    for i in collect(1:1:2^L)
        if map[i,2:3]==a
            k=i
            check+=1
        end
    end  
    if check==1
        return(k)
    else
        return("no")
    end
end



function ToBit(d,n)
    z=n
    a=zeros(Int64,z)
    i=0
    q=d
    while q>0
        a[n-i]=q%2
        q=(q-(q%2))/2
        i+=1
    end
    return(a)
end

#Hamming distance calculator

function h(a,b,L)
    h=0
    
    for i in collect(1:1:L)
        if a[i]!=b[i]
            h+=1
        end
    end
    return(h)
end



function add(L) ############ additve landscape generator 
    w=zeros(Float64,2^L)
    c=zeros(Float64,L)
    sig=zeros(Float64,L)
    w[1]=0.5
    
    for j in collect(1:1:L)
        c[j]=rand()-0.5
    end
    
    for i in collect(2:1:2^L)
        w[i]=w[1]
        for j in collect(1:1:L)
            sig[j]=ToBit(i-1,L)[j]-ToBit(0,L)[j]
            w[i]+=c[j]*sig[j]        
        end
    end
    
    wmin=minimum(w)
    epsilon=0  
    for n in collect(1:1:2^L) ######### Rescaling for w in 0 to 1
       w[n]=w[n]-wmin+epsilon ### for irreducinility of MS matrix
    end
            
    wmax=maximum(w)

    for n in collect(1:1:2^L) ######### Rescaling for w in 0 to 1
        w[n]=w[n]/wmax
    end

    return(w)
end

function hoc(L)
    w=zeros(Float64, 2^L)
    for i in collect(1:1:2^L)
        w[i]=rand()
    end
    
    return(w)   
end

function NK(L,k)
    w=zeros(Float64, 2^L)
    f=zeros(Float64, L,2^k)
    
    for j in collect(1:1:L)
        for l in collect(1:1:2^k)
            f[j,l]=rand() ### using adjacency model not block becasue that needs L to be divisible by k
        end
    end
    
    for i in collect(1:1:2^L)
        for j in collect(1:1:L)
            if j+k-1<=L
                w[i]+=f[j,ToDeci(ToBit(i-1,L)[j:j+k-1])+1]
            else
                w[i]+=f[j,ToDeci(vcat(ToBit(i-1,L)[j:L],ToBit(i-1,L)[1:k-1-(L-j)]))+1]
            end
        end
    end
    
    wmin=minimum(w)
            
           for n in collect(1:1:2^L) ######### Rescaling for w in 0 to 1
             w[n]=w[n]-wmin+0.001
            
            end
            
            wmax=maximum(w)
            
            for n in collect(1:1:2^L) ######### Rescaling for w in 0 to 1
               w[n]=w[n]/wmax
            end

    return(w)
end

function peaks(w,L)
    
    npeaks=1;
    wmax=maximum(w)
    
    for i in collect(1:1:2^L)
        c=0
        if w[i]!=wmax
            for j in collect(1:1:2^L)
                if h(ToBit(i-1,L),ToBit(j-1,L),L)==1 && w[i]>w[j]
                    c+=1
                end
            end
            if c==L
                npeaks+=1
            end
        end
    end

    return(npeaks)
end

function F(w,wopt,sigma,a,L) #Takes in binding aff and gives fitness
    F=zeros(Float64,a^L)
    for i in collect(1:1:a^L)
        F[i]=exp(-((w[i]-wopt)^2)/sigma^2)
    end
    Fmax=maximum(F)
    #return(F/Fmax)
    return(F)
end

function ToDeci(a)
    r=0
    L=length(a)
    for i in collect(1:1:length(a))
        r+=a[i]*2^(L-i)
    end
    return(r)
end

using LinearAlgebra

function MS(w,mu,L) ###### <ust be irreducible i.e. fitness values cannot be 0 or mu
    l=length(w)
    M=zeros(Float64,l,l)
    S=zeros(Float64,l,l)

    for i in collect(1:1:l)
        for j in collect(1:1:l)
            M[i,j]=mu^h(ToBit(i-1,L),ToBit(j-1,L),L)*(1-mu)^(L-h(ToBit(i-1,L),ToBit(j-1,L),L))
            S[i,i]=w[i]
        end
    end

    lambda=zeros(Float64,l,l)
    lambda=eigvecs(M*S)

    fin1=lambda[:,l]/sum(lambda[:,l])
    fin2=1/log(eigvals(M*S)[l]/eigvals(M*S)[l-1])

    return([fin1,fin2]) ## returning equilibirum state and rate of convergence
end


MS (generic function with 1 method)

# 1. NK landscapes 

## 1.1 Mean fitness at equilibrium 

In [9]:
L=2; tot=10000; sigma=0.1; mu=0.1;   #### Specify desired values

K=1;
range=collect(0:0.01:1)
dwm=zeros(Float64,tot,length(range),L)
dp=zeros(Float64,tot,length(range))

a=2


p=Array{Any,1}(undef,0)
for K in collect(1:1:L)
    for i in collect(1:1:tot)
            k=0
            w=NK(L,K)
            for wopt in range
                k+=1

           
                w1=F(w,1,sigma,a,L)
                w2=F(w,wopt,sigma,a,L)
               

                p1=peaks(w1,L)
                p2=peaks(w2,L)
                dp[i,k]=p2-p1

                wm_i=dot(MS(w1,mu,L)[1],w1)
                wm_f=dot(MS(w2,mu,L)[1],w2)
                dwm[i,k,K]=wm_f-wm_i
        end
    end
end



ddwm=zeros(Float64,length(range),L)
for K in collect(1:1:L)
    for k in collect(1:1:length(range))
        dwm[:,k,K]=dwm[:,k,K]-dwm[:,end,K]
        ddwm[k,K]=mean(dwm[:,k,K])
    end
end

writedlm("nk_L=$(L)_mu=$mu sigma=$sigma evol.txt",ddwm)


## 1.2 Adaptive walks

In [ ]:
### to calculate the number of steps of the walk and mean peak height reached 

L=3;   sigma=0.15; tot=10000
range=collect(0:0.01:1)


a=2

mean_steps=zeros(Float64,tot,length(range),L)
mean_peak_ht=zeros(Float64,tot, length(range),L)
var_peak_ht=zeros(Float64,tot, length(range),L)

b=0
for K in collect(1:1:L)
    b+=1; k=0


    for i in collect(1:1:tot)
            m=0; k+=1
            w=NK(L,K)
            for wopt in range
                m+=1

                w1=F(w,1,sigma,a,L) 
                w2=F(w,wopt,sigma,a,L)
                w2=w2/maximum(w2)

 
        neigh=Array{Any,1}(undef,0)
        options=Array{Any,1}(undef,0)
        step=Array{Any}(undef,0)
        steps=zeros(Float64,2^L)
        peak_ht=Array{Any}(undef,0)

        for j in collect(1:1:2^L)

            focal=ToBit(j-1,L)
            focal_index=j
            step=Array{Any}(undef,0)
            step=vcat(step,focal_index)
            stop=0

            while stop==0

                neigh=Array{Any,1}(undef,0)
                options=Array{Any,1}(undef,0)

                for k in collect(1:1:2^L)
                    if h(focal,ToBit(k-1,L),L)==1### including point mutations 
                        neigh=vcat(neigh, k)
                    end
                end

                for l in neigh
                    options=vcat(options,w2[l])
                end



                if   options!=[] && maximum(options)>=w2[focal_index]

                    for l in collect(1:1:L)
                         if maximum(options)==options[l] 

                            step=vcat(step,neigh[l])
                           
                            focal=ToBit(neigh[l]-1,L)
                            focal_index=neigh[l]
                        end
                    end 

                elseif options!=[] && maximum(options)<w2[focal_index]
                   #print(maximum(options)," ",land[focal_index])
                    stop+=1
                    peak_ht=vcat(peak_ht,w2[focal_index])

                end

            end
            
        
            steps[j]=length(step)


        end

        mean_steps[k,m,b]=mean(steps)
        mean_peak_ht[k,m,b]=mean(peak_ht)


        end
    end
    
end

writedlm("/Users/malvika/Desktop/mean_ht_L=$L.txt", mean_peak_ht)
writedlm("/Users/malvika/Desktop/mean_length_L=$L.txt", mean_steps)

In [ ]:
mean_peak_ht_av=zeros(Float64,length(rangee),L)

for b in collect(1:1:L)
    for i in collect(1:1:length(rangee))
        mean_peak_ht_av[i,b]=mean(mean_peak_ht[:,i,b])
    end

end


plot()
for K in collect(1:1:L)
    plot!(rangee, mean_peak_ht_av[:,K], label=:false,palette=:fruitpunch,linewidth=2)
end
plot!(xlabel=L"Optimal phenotype, $w_{opt}$", ylabel="Mean peak height reached", guidefontsize=13, guidefont=:Arial, tickfontsize=12, legend=:false)


In [ ]:
mean_steps_av=zeros(Float64,length(rangee),L)
L=5
for b in collect(1:1:L)
    for i in collect(1:1:length(rangee))
        mean_steps_av[i,b]=mean(mean_steps[:,i,b])
    end

end

plot()
for K in collect(1:1:L)
    plot!(rangee, mean_steps_av[:,K], label="K=$K",palette=:fruitpunch,linewidth=2)
end
plot!(xlabel=L"Optimal phenotype, $w_{opt}$", ylabel="Mean length of adaptive walk", guidefontsize=13, guidefont=:Arial, tickfontsize=14, legendfontsize=12)


# 2. Mismatch landsape 

## 2.1 Mean fitness at equilibrium

In [ ]:
######## chnage in peaks in mismatch mdoel ##############
L=8
sequences=Array{String,1}(undef,4^L)
c=1
for l1 in ["A","G","T","C"]
    for l2 in ["A","G","T","C"]
        for l3 in ["A","G","T","C"]
            for l4 in ["A","G","T","C"]
                for l5 in ["A","G","T","C"]
                    for l6 in ["A","G","T","C"]
                        for l7 in ["A","G","T","C"]
                            for l8 in ["A","G","T","C"]
                                
                                sequences[c]=l1*l2*l3*l4*l5*l6*l7*l8
                                c+=1
                            end
                        end
                    end
                end
            end
        end
    end
end



function F(w,wopt,sigma,l) #Takes in binding aff and gives fitness
    F=zeros(Float64,l)
    for i in collect(1:1:l)
        F[i]=exp(-((w[i]-wopt)^2)/sigma^2)
    end
    Fmax=maximum(F)
    #return(F/Fmax)
    return(F)
end

function MS(w,seq,mu,L) ###### <ust be irreducible i.e. fitness values cannot be 0 or mu
    l=length(w)
    M=zeros(Float64,l,l)
    S=zeros(Float64,l,l)
    
    for i in collect(1:1:l)
        for j in collect(1:1:l)
            M[i,j]=mu^h(seq[i],seq[j],L)*(1-mu)^(L-h(seq[i],seq[j],L))
            S[i,i]=w[i]
        end
    end
    
    lambda=zeros(Float64,l,l)
    lambda=eigvecs(M*S)
    
    fin1=lambda[:,l]/sum(lambda[:,l])
    fin2=1/log(eigvals(M*S)[l]/eigvals(M*S)[l-1])
    
    return([fin1,fin2]) ## returning equilibirum state and rate of convergence 
end

function rand_IC(l)
    r=zeros(Float64,l)
    for i in collect(1:1:l)
            r[i]=rand()
    end
    
    r=r/sum(r)
end

In [ ]:
tot=5000
L=8; a=4

P1=zeros(Float64,L+1,tot)
P2=zeros(Float64,L+1,tot)

P1[:,1]=rand_IC(L+1)
P2[:,1]=P1[:,1]

w=zeros(Float64,L+1)

epsilon=0.05;

for k in collect(0:1:L)
    w[k+1]=1-k*epsilon
end

####*********##################**********
      ### Strength of mutation 
 ### Strength of selection
####*********##################**********

g=0; gg=0


x=collect(0.02:0.02:0.12) ## mu
y=collect(0.1:0.1:1) ## sigma [0.05,0.1,0.15,0.2,0.25]#
    
best=zeros(Float64,length(y),length(x)) #### stores mismatch class that leads to highest mean equilibrium fitness
dwm=zeros(Float64,tot,L+1)

for sigma in y
    g+=1; gg=0
    for mu in x
        gg+=1
    
        w_m1=zeros(Float64,tot)
        w_m2=zeros(Float64,tot)
        dwm=zeros(Float64,tot,L+1)


        for kopt in collect(0:1:L)

            w1=F(w,1,sigma,L+1)
            w2=F(w,1-kopt*epsilon,sigma,L+1) 
            w_m1[1]=dot(w1,P1[:,1])
            w_m2[1]=dot(w2,P2[:,1])


            for t in collect(1:1:tot-1)

                w_m1[t+1]=dot(w1,P1[:,t])
                w_m2[t+1]=dot(w2,P2[:,t])
                dwm[t+1,kopt+1]= w_m2[t+1]-w_m1[t+1]

                for i in collect(1:1:L+1) ### MUTATION
                    d=i-1
                    P1[i,t+1]=((1-mu)+mu*(a-2)*d/((a-1)*L))*P1[i,t]
                    P2[i,t+1]=((1-mu)+mu*(a-2)*d/((a-1)*L))*P2[i,t]

                    if i!=L+1
                        P1[i,t+1]+=mu*(d+1)/((a-1)*L)*P1[i+1,t]
                        P2[i,t+1]+=mu*(d+1)/((a-1)*L)*P2[i+1,t]
                    end

                     if i!=1
                        P1[i,t+1]+=mu*(L-(d-1))/(L)*P1[i-1,t]
                        P2[i,t+1]+=mu*(L-(d-1))/(L)*P2[i-1,t]
                    end
                end


               for i in collect(1:1:L+1) ### SELECTION
                 P1[i,t+1]=w1[i]*P1[i,t+1] 
                 P2[i,t+1]=w2[i]*P2[i,t+1] 
              end

               P1[:,t+1]=P1[:,t+1]/sum(P1[:,t+1])
               P2[:,t+1]=P2[:,t+1]/sum(P2[:,t+1])




            end

        end
        for k in collect(0:1:L)
            if dwm[tot,k+1]==maximum(dwm[tot,:])
                best[g,gg]=k
            end
        end
    end
        
end

In [ ]:
# Plotting change in mean fitness for a specific mu and sigma

plot(collect(0:1:L),dwm[end,:],linecolor=:black,linewidth=2)#,label="mu=$mu, sigma=$sigma", palette =:RdYlGn_9)
scatter!(collect(0:1:L),dwm[end,:],legend=:false,markercolor=:black,markersize=5)
plot!(xlabel=L"$\sigma$", ylabel="Change in mean fitness at equilibrium", guidefontsize=15, guidefont=:Arial, tickfontsize=15, legendfontsize=12)

In [ ]:
# Plotting heatmap of the "best" mismatch class

data = best
c=heatmap(x,y, data,c=cgrad([:black,:white]), xlabel=L"\mu", ylabel=L"\sigma", guidefontsize=15,  tickfontsize=12, legendfontsize=12)

## 2.2 Adaptive walks

In [ ]:
L=8; a=4

av_steps=zeros(Float64,L+1)

for dopt in collect(0:1:L)


    av_steps1=0; av_steps2=0
    dr=0; 

    for d in collect(0:1:dopt)
        av_steps1+=(a-1)^(d) * factorial(L)/(factorial(d)*factorial(L-d))*(dopt-d)
        dr+=(a-1)^d * factorial(L)/(factorial(d)*factorial(L-d))
    end

    for d in collect(dopt+1:1:L)   
        av_steps2+=(a-1)^(d) * factorial(L)/(factorial(d)*factorial(L-d))*(-dopt+d)
        dr+=(a-1)^(d) * factorial(L)/(factorial(d)*factorial(L-d))
    end
    
         
        av_steps[dopt+1]=av_steps1/dr+av_steps2/dr
    
end


plot(["8","7","6","5","4","3","2","1","0"],[av_steps[end],av_steps[end-1],av_steps[end-2],av_steps[end-3],av_steps[end-4],av_steps[end-5],av_steps[end-6],av_steps[end-7],av_steps[end-8]],   linecolour=:black,linewidth=2)
scatter!(["8","7","6","5","4","3","2","1","0"], [av_steps[end],av_steps[end-1],av_steps[end-2],av_steps[end-3],av_steps[end-4],av_steps[end-5],av_steps[end-6],av_steps[end-7],av_steps[end-8]],   markercolour=:black,markersize=5,legend=:false)
plot!(guidefontsize=14, tickfontsize=14) ########### Reverse x axis label

# 3. Empirical landscapes

## 3.1 Mean fitness at equilibrium

In [ ]:
using Random, Distributions

function F(w,wopt,sigma,l) #Takes in binding aff and gives fitness
    F=zeros(Float64,l)
    for i in collect(1:1:l)
        F[i]=exp(-((w[i]-wopt)^2)/sigma^2)
    end
    Fmax=maximum(F)
    #return(F/Fmax)
    return(F)
end

function h(a,b,L)
    h=0

    for i in collect(1:1:L)
        if a[i]!=b[i]
            h+=1
        end
    end
    return(h)
end

function indel(a,b,L) ### takes in seqs and returns 1 if they differ by small indel else returns 0
   if h(a,b,L)!=1
        c=0; d=0;

        for i in collect(1:1:L-1)
            if a[i]==b[i+1]
                c+=1
            end
        end

        for i in collect(2:1:L)
            if a[i]==b[i-1]
                d+=1
            end
        end

        if c==L-1 || d==L-1
            return(1)
        else
            return(100)
        end
    else
        return(1)

   end
end


function rc(a)
    l=length(a)
    b=Array{Any,1}(undef,l)
    for i in collect(1:1:l)
        if a[i]=='A'
            b[l-i+1]='T'
        elseif a[i]=='T'
            b[l-i+1]='A'
         elseif a[i]=='G'
            b[l-i+1]='C'
         elseif a[i]=='C'
            b[l-i+1]='G'
        end
    end
    return(b)
end




function MS_indel_rc(w,seq,mu,L) ####MS w/ indels ###### <ust be irreducible i.e. fitness values cannot be 0 or mu
    l=length(w)
    M=zeros(Float64,l,l)
    S=zeros(Float64,l,l)

    for i in collect(1:1:l)
        for j in collect(1:1:l)
            d=minimum([h(seq[i],seq[j],L),indel(seq[i],seq[j],L),h(rc(seq[i]),(seq[j]),L),indel(rc(seq[i]),seq[j],L)]) ### rate of indel==rate of point mu
            M[i,j]=mu^d*(1-mu)^(L-d)
            S[i,i]=w[i]
        end
    end

    lambda=zeros(Float64,l,l)
    lambda=eigvecs(M*S)

    fin1=lambda[:,l]/sum(lambda[:,l])
    fin2=1/log(eigvals(M*S)[l]/eigvals(M*S)[l-1])

    return([fin1,fin2]) ## returning equilibirum state and rate of convergence
end

function rand_IC(l)
    r=zeros(Float64,l)
    for i in collect(1:1:l)
            r[i]=rand()
    end

    r=r/sum(r)
end

#####################################

In [ ]:

L=8
mu=0.1 ; sigma=0.15 ; #### Specify desired values 

noise=0 ## specify whether to add noise in the data

wm=zeros(Float64,1137,7)


global count=0

for wopt in collect(0.35:0.025:0.5)
    global count+=1
    println(wopt)
    wm_f=zeros(Float64,1137)
    k=1
    for i in collect(1:1:1146)
        println(i)
        if i!=1035 && i!=1036 && i!=1037 && i!=1013 && i!=1038 && i!=1039 && i!=1040 && i!=1058 && i!=1077
                if i==1
                    start=2
                else
                    start=3
                end
                
                data=readdlm("TF_wopt=$wopt _sigma=$sigma _$i.tsv")
                #w=data[start:end,2]

                dom=readdlm("results_TF_$i/set1_genotype_measures.csv")[2:end]
                len=length(dom)


                if data[2,1]==data[3,1]
                    start=3
                else
                    start=2
                end

                land=Array{Any,1}(undef,0)
                seqs=(dom)
                delta=Array{Any,1}(undef,0)

                for i in collect(1:1:len)
                    for j in collect(start:1:length(data[1:end,1]))
                        if seqs[i]==data[j,1]
                            land=vcat(land,data[j,2])
                            delta=vcat(delta,data[j,3])
                        end
                    end
                end


                w=land
                l=length(w)

                if noise==1
                  for b in collect(1:1:l)
                    n=Normal(0,delta[b]/3) ## Noise, assuming delta is 3 sd
                    d=rand(n)
                    w[b]=w[b]+d
                  end
                end

                wm_f[k]=dot(MS_indel_rc(w,seqs,mu,L)[1],w)
                wm[k,count]=wm_f[k]
                k+=1
        end
    end

end

writedlm("noise=$noise_wm_sigma=$sigma mu=$mu.txt",wm)


In [ ]:
### Plotting the heatmap #######

y=[ 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]#, 0.6, 0.8, 1.0]#collect(0.2:0.2:0.6)
x=[0.001,0.0025,0.005,0.01,0.025,0.05,0.075]#[0.01,0.025,0.05,0.075]#, 0.01, 0.025,0.05,0.075,0.1] [0.001, 0.0025, 0.005,
best=zeros(Float64,length(x),length(y))
g=0
wm_mean=zeros(Float64,7)

for sigma in y
    g+=1; gg=0
    for mu in x
        gg+=1
        wm=readdlm("noise=$noise_wm_sigma=$sigma mu=$mu.txt")
        for i in collect(1:1:7)
            wm_mean[i]=mean(wm[:,i])
        end
        best[gg,g]=argmax(wm_mean)
    end
end

for i in collect(1:1:length(x))
    for j in collect(1:1:length(y))
        best[i,j]=0.35+0.025*(best[i,j]-1)
    end
end


for i in collect(1:1:length(x))
    x[i]=log10(x[i])
end


data = transpose(best)
Plots.heatmap(x,
   y, data,
    c=cgrad([:white, :black]),
    xlabel=L" \mu", ylabel=L"\sigma",
   guidefontsize=15,  tickfontsize=12, legendfontsize=12)

## 3.2 Adaptive walks

In [ ]:
function rc(a)
    l=length(a)
    b=Array{Any,1}(undef,l)
    for i in collect(1:1:l)
        if a[i]=='A'
            b[l-i+1]='T'
        elseif a[i]=='T'
            b[l-i+1]='A'
         elseif a[i]=='G'
            b[l-i+1]='C'
         elseif a[i]=='C'
            b[l-i+1]='G'
        end
    end
    return(b)
end

function indel(a,b,L) ### takes in seqs and returns 1 if they differ by small indel else returns 0
   if h(a,b,L)!=1
        c=0; d=0;

        for i in collect(1:1:L-1)
            if a[i]==b[i+1]
                c+=1
            end
        end

        for i in collect(2:1:L)
            if a[i]==b[i-1]
                d+=1
            end
        end

        if c==L-1 || d==L-1
            return(1)
        else
            return(0)
        end

   end
end

list=zeros(Int64,1137)
global k=1
for i in collect(1:1:1146)
    if i!=1035 && i!=1036 && i!=1037 && i!=1013 && i!=1038 && i!=1039 && i!=1040 && i!=1058 && i!=1077
        list[k]=i
        global k+=1
    end
end

In [ ]:
sigma=0.15
wopt=0.5
L=8;
tot=1137

mean_steps=zeros(Float64,tot)
mean_peak_ht=zeros(Float64,tot)
var_peak_ht=zeros(Float64,tot)
glob_peak=zeros(Float64,tot)
glob_peak_ht=0

global m=1

for i in list
    
    data=readdlm("TF_wopt=$wopt _sigma=0.15 _$i.tsv")
    result=readdlm("results_wopt=$(wopt)_sigma=$(sigma)_$i.txt")
    dom=readdlm("results_TF_$i/set1_genotype_measures.csv")[2:end] ### Results of Genonets analysis on landscapes to get the domiannt network

    len=length(dom)


    if data[2,1]==data[3,1]
        start=3
    else
        start=2
    end

    seqs=(dom)
    land=Array{Any,1}(undef,0)
    delta=Array{Any,1}(undef,0)


    for i in collect(1:1:len)
        for j in collect(start:1:length(data[1:end,1]))
            if seqs[i]==data[j,1]
                land=vcat(land,data[j,2])
                delta=vcat(delta,data[j,3])
            end
        end
    end

    globpeakseqs=Array{String,1}(undef,0)
    globpeakhts=Array{Float64,1}(undef,0)
    peakseqs=Array{String,1}(undef,0)
    peakhts=Array{Float64,1}(undef,0)
    stop=0
    
    for j in collect(1:1:length(result))
            if result[j]==1
                stop=1
           end
            for l in collect(start:1:length(data[:,1]))
                if stop==0
                    if result[j]==data[l,1]
                            globpeakseqs=vcat(globpeakseqs,data[l,1])
                            globpeakhts=vcat(globpeakhts,data[l,2])
                    end
                else
                    if result[j]==data[l,1]
                            peakseqs=vcat(peakseqs,data[l,1])
                            peakhts=vcat(peakhts,data[l,2])
                    end
                end
            end

    end


    glob_peak_ht=mean(globpeakhts)

    neigh=Array{Any,1}(undef,0)
    options=Array{Any,1}(undef,0)
    step=Array{Any}(undef,0)
    steps=Array{Any,1}(undef,0)#zeros(Float64,len)
    peak_ht_reached=Array{Any}(undef,0)
    seq_reached=Array{Any}(undef,0)

    for j in collect(1:1:len)
      
        focal=seqs[j]
        focal_index=j
        step=Array{Any}(undef,0)
        if focal in peakseqs || focal in globpeakseqs ### Not starting from peak sequnces 
                arrival=1; stop=1
               
        else
            step=vcat(step,focal)
            stop=0
            arrival=0
        end
      
        while stop==0

              if focal in peakseqs || focal in globpeakseqs ### Not starting from peak sequnces 
                arrival=1; stop=1
              end

           
            neigh=Array{Any,1}(undef,0)
            options=Array{Any,1}(undef,0)

            for k in collect(1:1:len)
              if h(focal,seqs[k],L)==1 || indel(rc(focal),seqs[k],L)==1 || h(rc(focal),seqs[k],L)==1 || indel(focal,seqs[k],L)==1 ### including point mutations and indels
                    if land[k]>land[focal_index]-delta[focal_index] ## only adding "fitter" neighbours    
                        if seqs[k] in step ## not allowing backsteps
                        else        
                            neigh=vcat(neigh, k)
                           options=vcat(options,land[k])
                        end
                    end
                end
            end

         
            if  length(options)!=0 && arrival==0
                if  maximum(options)>land[focal_index]  ## If strictly larger then pick max
                    for l in collect(1:1:length(neigh))
                            if maximum(options)==options[l]
                              step=vcat(step,seqs[neigh[l]])
                              focal=seqs[neigh[l]]
                              focal_index=neigh[l]
                            end
                      end
                  else
              
                    pick =  rand(neigh)   ## Else pick a random neighbour
                    step=vcat(step,seqs[pick])
                    focal=seqs[pick]
                    focal_index=pick
                  end

            else 
                stop+=1
                peak_ht_reached=vcat(peak_ht_reached,land[focal_index]/glob_peak_ht)
                seq_reached=vcat(seq_reached,focal)

            end

        end

  
        if length(step)!=0 ### starting from non peak seqs
            steps=vcat(steps,length(step))
           
        end
       

    end



    glob=0

    for term_seq in seq_reached
        if term_seq in globpeakseqs
            glob+=1
        end
    end

    

    if length(steps)!=0
        mean_steps[m]=mean(steps)
        glob=glob/length(peak_ht_reached)
        println(glob)
        glob_peak[m]=glob
        mean_peak_ht[m]=mean(peak_ht_reached)
    else
        mean_steps[m]=0
        mean_peak_ht[m]=glob_peak_ht
        glob_peak[m]=1
    end

    

    global m+=1

end

writedlm("mean_steps_$(wopt).txt",mean_steps)
writedlm("mean_peakht_$(wopt).txt",mean_peak_ht)
writedlm("glob_peakht_$(wopt).txt",glob_peak)


In [ ]:
k=0
for wopt in collect(0.35:0.025:0.5)
        mean_steps=readdlm("mean_steps_$(wopt).txt")
end


violin([" 0.35 " " 0.375 " " 0.4 " " 0.425 " " 0.45 " " 0.475 " " 0.5 "],mean_steps,legend=:false,fillcolor=:lightgrey,fillalpha=0.6,linewidth=1,linecolor=:black)#fillalpha=0.4)
boxplot!([ " 0.35 " " 0.375 " " 0.4 " " 0.425 " " 0.45 " " 0.475 " " 0.5 "],mean_steps,legend=:false,fillcolor=:white,fillalpha=0.5,markeralpha=0,linewidth=1,linecolor=:black, ylims=(1.5, 5))
plot!(guidefontsize=13, tickfontsize=14,legend=:false)#, xlabel="Optimal binding affinity",ylabel="Average length of an adaptive walk")